In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import importlib

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

pd.set_option('display.max_columns', 200)
plt.style.use('ggplot')

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

sys.path.append('/Users/i.tolkachev/freelance/git/softpro/')
import odds

from tqdm import tqdm_notebook

tqdm_notebook().pandas()

In [4]:
data_1 = pd.read_excel('/Users/i.tolkachev/freelance/softpro/20191117_19.xlsx', sheet_name='all')
data_2 = pd.read_excel('/Users/i.tolkachev/freelance/softpro/20191123.xlsx', sheet_name='Sheet2')
data_3 = pd.read_excel('/Users/i.tolkachev/freelance/softpro/20191125.xlsx', sheet_name='Sheet2')

In [5]:
data_1.shape, data_2.shape, data_3.shape

((190873, 529), (176564, 529), (36248, 529))

In [6]:
data_1_nodups = data_1.drop(['Id', 'DateAdded'], axis=1).drop_duplicates()
data_2_nodups = data_2.drop(['Id', 'DateAdded'], axis=1).drop_duplicates()
data_3_nodups = data_3.drop(['Id', 'DateAdded'], axis=1).drop_duplicates()

data_1_nodups.shape, data_2_nodups.shape, data_3_nodups.shape

((3459, 527), (4142, 527), (892, 527))

In [52]:
columns_data = [c for c in data_3.columns.tolist() if not ('computed' in c or 'disc' in c)]

ml_columns = ['HOME', 'DRAW', 'AWAY']
total_columns = [col for col in columns_data if 'TO' in col or 'TU' in col]
total_home_columns = [col for col in columns_data if 'T1O' in col or 'T1U' in col]
total_away_columns = [col for col in columns_data if 'T2O' in col or 'T2U' in col]
hdp_columns = [col for col in columns_data if 'HDP' in col]

In [8]:
book_odds_ordered = pd.concat([data_1_nodups, data_2_nodups, data_3_nodups], ignore_index=True)
book_odds_ordered.shape

(8493, 527)

In [13]:
def fair_odd(odd, margin):
    return (odd*(margin+1))

def margin_2(odd_1, odd_2):
    return 1/odd_1 + 1/odd_2 - 1

def margin_3(odd_1, odd_2, odd_3):
    return 1/odd_1 + 1/odd_2 + 1/odd_3 - 1

In [31]:
filled_data

,EventId,IsLive,StartTime,HomeTeam,AwayTeam,HOME,DRAW,AWAY,TO_2_25,TU_2_25,TO_2_5,TU_2_5,TO_2_75,TU_2_75,TO_3_0,TU_3_0,TO_3_25,TU_3_25,T1O_1_5,T1U_1_5,T2O_1_5,T2U_1_5,"HOME_HDP 0,0","AWAY_HDP 0,0","HOME_HDP -0,25","AWAY_HDP 0,25","HOME_HDP -0,5","AWAY_HDP 0,5","HOME_HDP -0,75","AWAY_HDP 0,75","HOME_HDP -1,0","AWAY_HDP 1,0"
0,1063863259,False,2019-11-17 12:15:00,Pelikan Lowicz,RKS Lechia Tomaszow Mazowiecki,1.813,3.74,4.08,1.456,2.68,1.649,2.22,1.833,1.98,2.1,1.724,2.4,1.558,1.724,2.11,1.724,2.11,1.349,3.11,1.578,2.36,1.813,2.0,2.06,1.757,2.48,1.526


In [84]:
# main function for computing errors
from collections import defaultdict

total_errors = defaultdict(list)
int_total_errors = defaultdict(list)
half_total_errors = defaultdict(list)
quarter_total_errors = defaultdict(list)

total_1_errors = defaultdict(list)
int_total_1_errors = defaultdict(list)
half_total_1_errors = defaultdict(list)
quarter_total_1_errors = defaultdict(list)

total_2_errors = defaultdict(list)
int_total_2_errors = defaultdict(list)
half_total_2_errors = defaultdict(list)
quarter_total_2_errors = defaultdict(list)

hdp_1_errors = defaultdict(list)
int_hdp_1_errors = defaultdict(list)
half_hdp_1_errors = defaultdict(list)
quarter_hdp_1_errors = defaultdict(list)

ml_errors = defaultdict(list)


for i in tqdm_notebook(range(book_odds_ordered.shape[0])):
    try:
        filled_data = book_odds_ordered.iloc[[i], :].dropna(axis=1).reset_index()
        print('input odds before game started', filled_data.shape)
#         print(filled_data.head())
        
        not_empty_columns = [c for c in filled_data.columns if not ('computed' in c or 'disc' in c)]
        filled_data = filled_data[not_empty_columns]

        ml_columns = ['HOME', 'DRAW', 'AWAY']
        total_columns = [col for col in not_empty_columns if 'TO' in col or 'TU' in col]
        total_home_columns = [col for col in not_empty_columns if 'T1O' in col or 'T1U' in col]
        total_away_columns = [col for col in not_empty_columns if 'T2O' in col or 'T2U' in col]
        hdp_columns = [col for col in not_empty_columns if 'HDP' in col]
        
#         print(total_columns)
        # compute model init parameters
        half_total_over = [c for c in total_columns 
                           if ('TO' in c) 
                           and ('_5' in c) 
                           and (filled_data[c][0] > 1.5) 
                           and (filled_data[c][0] < 3)
                          ][0]
        half_total_value = float(half_total_over[3:].replace('_', '.'))
        half_total_init = [half_total_over, half_total_over.replace('O', 'U')]

        half_hdp_home = [c for c in hdp_columns 
                         if ('HOME_HDP' in c)
                         and (',5' in c) 
                         and (filled_data[c][0] > 1.5) 
                         and (filled_data[c][0] < 3)
                        ][0]
        half_hdp_value = float(half_hdp_home[8:].replace(',', '.'))
        half_hdp_value_name = (-1)*half_hdp_value if half_hdp_value != 0 else half_hdp_value
        half_hdp_init = [half_hdp_home, f'AWAY_HDP {half_hdp_value_name}'.replace('.', ',')]

        # margin
        half_total_margin = margin_2(
            *filled_data[half_total_init].values[0].tolist()
        )

        half_hdp_margin = margin_2(
            *filled_data[half_hdp_init].values[0].tolist()
        )
        
#         print('total margin:', half_total_margin)
#         print('hdp margin:', half_hdp_margin)

        to, tu = filled_data[half_total_init].values[0].tolist()
        fair_to, fair_tu = fair_odd(filled_data[half_total_init], half_total_margin).values[0].tolist()

        hdp1, hdp2 = filled_data[half_hdp_init].values[0].tolist()
        fair_hdp1, fair_hdp2 = fair_odd(filled_data[half_hdp_init], half_hdp_margin).values[0].tolist()

        print(half_total_value, to, tu, half_hdp_value, hdp1, hdp2)
        
        book = 'pinnacle'
        try:
            result_odds, mu, mu1, mu2 = odds.compute_odds(half_total_value, to, tu,
                                                          half_hdp_value, hdp1, hdp2)

#             print('TO fair: ', fair_to, 'TU fair: ', fair_tu)
#             to_pred = result_odds[f'T O {half_total_value}']
#             tu_pred = result_odds[f'T U {half_total_value}']
#             print('TO pred: ', to_pred, 'TU fair: ', tu_pred)

#             plt.figure(figsize=(10, 10))
            x, y, y_pred = [], [], []
            # totals errors
            for i in range(len(total_columns[::2])):
                to, tu = filled_data[total_columns[2*i:2*i+2]].values[0].tolist()
                t_val = float(total_columns[2*i][3:].replace('_', '.'))
                margin = margin_2(to, tu)
                fair_to = fair_odd(to, margin)
                fair_tu = fair_odd(tu, margin)
#                 print('TO: ', to, 'TU: ', tu, 'margin', margin)
#                 print('T value: ', t_val)
#                 print('TO fair: ', fair_to, 'TU fair: ', fair_tu)
                to_pred = result_odds[f'T O {t_val}']
                tu_pred = result_odds[f'T U {t_val}']
#                 print('TO pred: ', to_pred, 'TU fair: ', tu_pred)
#                 print('Error TO: ', abs(odds.odd_to_prob(fair_to) - odds.odd_to_prob(to_pred)))
#                 print('Error TU: ', abs(odds.odd_to_prob(fair_tu) - odds.odd_to_prob(tu_pred)))
                x.append(t_val)
                y.append(odds.odd_to_prob(fair_tu))
                y_pred.append(odds.odd_to_prob(tu_pred))
                
                total_errors[book].append(abs(odds.odd_to_prob(fair_tu) - odds.odd_to_prob(tu_pred)))
                if t_val % 1 == 0.5:
                    half_total_errors[book].append(abs(odds.odd_to_prob(fair_tu) - odds.odd_to_prob(tu_pred)))
                elif t_val % 1 == 0.0:
                    int_total_errors[book].append(abs(odds.odd_to_prob(fair_tu) - odds.odd_to_prob(tu_pred)))
                else:
                    quarter_total_errors[book].append(abs(odds.odd_to_prob(fair_tu) - odds.odd_to_prob(tu_pred)))

#             plt.plot(x, y, 'o', label="fair TU", alpha=0.7)
#             plt.plot(x, y_pred, 'o', label="pred TU", alpha=0.7)

#             # plot poisson distr
#             x, y = [], []
#             for x0 in range(0, 7):
#                 x.append(x0+0.5)
#                 p_opt = odds.PoissonOptimizer(x0, 0.5)
#                 y.append(p_opt.sum_pmf(mu))
#             plt.plot(x, y, 'o-', label="poisson pmf", alpha=0.7)
#             plt.legend(loc='upper left')
#             plt.show()

            # totals home errors
            for i in range(len(total_home_columns[::2])):
                to, tu = filled_data[total_home_columns[2*i:2*i+2]].values[0].tolist()
                t_val = float(total_home_columns[2*i][4:].replace('_', '.'))
                margin = margin_2(to, tu)
#                 print('total margin:', margin, to, tu)
                fair_to = fair_odd(to, margin)
                fair_tu = fair_odd(tu, margin)
#                 print('T1O: ', to, 'T1U: ', tu, 'margin', margin)
#                 print('T value: ', t_val)
#                 print('T1O fair: ', fair_to, 'T1U fair: ', fair_tu)
                to_pred = result_odds[f'T1 O {t_val}']
                tu_pred = result_odds[f'T1 U {t_val}']
#                 print('T1O pred: ', to_pred, 'T1U fair: ', tu_pred)
#                 print('Error T1O: ', abs(odds.odd_to_prob(fair_to) - odds.odd_to_prob(to_pred)))
#                 print('Error T1U: ', abs(odds.odd_to_prob(fair_tu) - odds.odd_to_prob(tu_pred)))
#                 print()
    
                total_1_errors[book].append(abs(odds.odd_to_prob(fair_tu) - odds.odd_to_prob(tu_pred)))
                if t_val % 1 == 0.5:
                    half_total_1_errors[book].append(abs(odds.odd_to_prob(fair_tu) - odds.odd_to_prob(tu_pred)))
                elif t_val % 1 == 0.0:
                    int_total_1_errors[book].append(abs(odds.odd_to_prob(fair_tu) - odds.odd_to_prob(tu_pred)))
                else:
                    quarter_total_1_errors[book].append(abs(odds.odd_to_prob(fair_tu) - odds.odd_to_prob(tu_pred)))
#                 total_1_errors.append(abs(odds.odd_to_prob(fair_tu) - odds.odd_to_prob(tu_pred)))

#             # totals away errors
#             for i in range(len(total_away_columns[::2])):
#                 to, tu = filled_data[total_away_columns[2*i:2*i+2]].values[0].tolist()
#                 t_val = float(total_away_columns[2*i][4:].replace('_', '.'))
#                 margin = margin_2(to, tu)
#                 fair_to = fair_odd(to, margin)
#                 fair_tu = fair_odd(tu, margin)
# #                 print('T2O: ', to, 'T2U: ', tu, 'margin', margin)
# #                 print('T value: ', t_val)
# #                 print('T2O fair: ', fair_to, 'T2U fair: ', fair_tu)
#                 to_pred = result_odds[f'T2 O {t_val}']
#                 tu_pred = result_odds[f'T2 U {t_val}']
# #                 print('T2O pred: ', to_pred, 'T2U fair: ', tu_pred)
# #                 print('Error T2O: ', abs(odds.odd_to_prob(fair_to) - odds.odd_to_prob(to_pred)))
# #                 print('Error T2U: ', abs(odds.odd_to_prob(fair_tu) - odds.odd_to_prob(tu_pred)))
# #                 total_2_errors.append(abs(odds.odd_to_prob(fair_to) - odds.odd_to_prob(to_pred)))
#                 total_2_errors[book].append(abs(odds.odd_to_prob(fair_tu) - odds.odd_to_prob(tu_pred)))
#                 if t_val % 1 == 0.5:
#                     half_total_2_errors[book].append(abs(odds.odd_to_prob(fair_tu) - odds.odd_to_prob(tu_pred)))
#                 elif t_val % 1 == 0.0:
#                     int_total_2_errors[book].append(abs(odds.odd_to_prob(fair_tu) - odds.odd_to_prob(tu_pred)))
#                 else:
#                     quarter_total_2_errors[book].append(abs(odds.odd_to_prob(fair_tu) - odds.odd_to_prob(tu_pred)))
            
            # handicap errors
            for home_col in [c for c in hdp_columns if 'HOME' in c]:
                hdp_value = float(home_col.split(' ')[1].replace(',','.'))
                hdp_value_name = (-1)*hdp_value if hdp_value != 0 else hdp_value
                away_col = f'AWAY_HDP {hdp_value_name}'.replace('.', ',')
#                 print(home_col, away_col, hdp_value)
                try:
                    hdp_home, hdp_away = filled_data[[home_col, away_col]].values[0].tolist()
                except:
                    print('no alternative odd')
                    continue
                margin = margin_2(hdp_home, hdp_away)
#                 print('hdp margin:', margin)
                fair_home_hdp = fair_odd(hdp_home, margin)
                fair_away_hdp = fair_odd(hdp_away, margin)
#                 print('T2O: ', to, 'T2U: ', tu, 'margin', margin)
#                 print('T value: ', t_val)
#                 print('T2O fair: ', fair_to, 'T2U fair: ', fair_tu)
                home_hdp_pred = result_odds[f'HDP1 {hdp_value}']
#                 away_hdp_pred = result_odds[f'HDP2 {(-1)*hdp_value}']
                
                hdp_1_errors[book].append(abs(odds.odd_to_prob(fair_home_hdp) - odds.odd_to_prob(home_hdp_pred)))
                if t_val % 1 == 0.5:
                    half_hdp_1_errors[book].append(abs(odds.odd_to_prob(fair_home_hdp) - odds.odd_to_prob(home_hdp_pred)))
                elif t_val % 1 == 0.0:
                    int_hdp_1_errors[book].append(abs(odds.odd_to_prob(fair_home_hdp) - odds.odd_to_prob(home_hdp_pred)))
                else:
                    quarter_hdp_1_errors[book].append(abs(odds.odd_to_prob(fair_home_hdp) - odds.odd_to_prob(home_hdp_pred)))

    
            # ML
            home, draw, away = filled_data[ml_columns].values[0].tolist()
            margin = margin_3(home, draw, away)

            print('ML :', home, draw, away)
#             print('ML margin:', margin)
            fair_home = fair_odd(home, margin)
            home_pred = result_odds[f'ML 1']
#             print('Home fair: ', fair_home, 'Home pred: ', home_pred)
#             print('Error home: ', abs(odds.odd_to_prob(fair_home) - odds.odd_to_prob(home_pred)))
            ml_errors[book].append(abs(odds.odd_to_prob(fair_home) - odds.odd_to_prob(home_pred)))

            fair_draw = fair_odd(draw, margin)
            draw_pred = result_odds[f'ML X']
#             print('Draw fair: ', fair_draw, 'Draw pred: ', draw_pred)
#             print('Error draw: ', abs(odds.odd_to_prob(fair_draw) - odds.odd_to_prob(draw_pred)))
            ml_errors[book].append(abs(odds.odd_to_prob(fair_draw) - odds.odd_to_prob(draw_pred)))
        
            fair_away = fair_odd(away, margin)
            away_pred = result_odds[f'ML 2']
#             print('Away fair: ', fair_away, 'Away pred: ', away_pred)
#             print('Error away: ', abs(odds.odd_to_prob(fair_away) - odds.odd_to_prob(away_pred)))
            ml_errors[book].append(abs(odds.odd_to_prob(fair_away) - odds.odd_to_prob(away_pred)))

#             # plot skellam distr
#             plt.figure(figsize=(10, 10))
#             x, y = [], []
#             for x0 in range(-6, 7):
#                 x.append(x0)
#                 s_opt = odds.SkellamOptimizer(x0, 0.5, mu)
#                 y.append(s_opt.pmf(x0, mu2))
#             plt.plot(x, y, 'o-', label="skellam pmf", alpha=0.7)
#             plt.scatter([0], [odds.odd_to_prob(fair_draw)], label="fair draw", alpha=0.7)
#             plt.scatter([0], [odds.odd_to_prob(draw_pred)], label="pred draw", alpha=0.7)
#             plt.legend(loc='upper left')
#             plt.show()
#             break
        except StopIteration as e:
            print(e)
#             break
    except Exception as e:
        print('next', e)
#         break

input odds before game started (1, 87)
2.5 1.649 2.22 -0.5 1.813 2.0
ML : 1.813 3.74 4.08
input odds before game started (1, 81)
3.5 1.751 2.08 2.5 1.735 2.13
ML : 17.47 8.87 1.12
input odds before game started (1, 87)
2.5 2.2 1.675 -0.5 2.14 1.729
ML : 2.14 3.33 3.53
input odds before game started (1, 87)
2.5 2.24 1.625 -0.5 2.35 1.571
ML : 2.36 3.13 3.15
input odds before game started (1, 60)
4.5 1.8 2.01 -3.5 2.19 1.68
next "['HOME' 'DRAW' 'AWAY'] not in index"
input odds before game started (1, 81)
2.5 1.8 2.01 -1.5 2.3 1.609
ML : 1.367 4.58 7.85
input odds before game started (1, 105)
2.5 2.59 1.471 -0.5 2.34 1.598
ML : 2.35 2.99 3.44
input odds before game started (1, 105)
2.5 2.38 1.581 -0.5 1.826 2.02


/Users/i.tolkachev/freelance/git/softpro/optimizers.py:110: RuntimeWarning: invalid value encountered in sqrt
  return np.exp(-self.mu) * ((self.mu - mu2) / mu2) ** (k / 2) * iv(k, 2 * np.sqrt((self.mu - mu2) * mu2))
/Users/i.tolkachev/freelance/git/softpro/optimizers.py:110: RuntimeWarning: invalid value encountered in power
  return np.exp(-self.mu) * ((self.mu - mu2) / mu2) ** (k / 2) * iv(k, 2 * np.sqrt((self.mu - mu2) * mu2))


ML : 1.826 3.34 4.87
input odds before game started (1, 81)
3.5 1.751 2.08 1.5 2.02 1.833
ML : 7.93 6.16 1.288
input odds before game started (1, 81)
3.5 2.19 1.666 -1.5 1.546 2.43
ML : 1.154 6.91 14.71
input odds before game started (1, 111)
2.5 2.12 1.729 -0.5 2.29 1.636
ML : 2.29 3.31 3.21
input odds before game started (1, 105)
1.5 1.568 2.41 -0.5 2.14 1.729
ML : 2.14 3.06 3.89
input odds before game started (1, 105)
2.5 2.54 1.515 -0.5 1.943 1.909
ML : 1.943 3.25 4.42
input odds before game started (1, 87)
2.5 2.33 1.595 -0.5 2.66 1.462
ML : 2.66 3.18 2.71
input odds before game started (1, 66)
4.5 1.99 1.84 -3.5 1.943 1.9
next "['HOME' 'DRAW' 'AWAY'] not in index"
input odds before game started (1, 87)
2.5 2.42 1.537 -0.5 2.42 1.54
ML : 2.43 3.02 3.16
input odds before game started (1, 105)
2.5 2.33 1.602 -1.5 2.56 1.52
ML : 1.395 4.18 9.4
input odds before game started (1, 111)
2.5 2.23 1.657 -0.5 2.57 1.515
ML : 2.57 3.22 2.85
input odds before game started (1, 87)
2.5 2.24 1.6

ML : 1.684 5.08 3.62
input odds before game started (1, 87)
3.5 1.847 1.961 -0.5 1.746 2.07
ML : 1.746 4.92 3.44
input odds before game started (1, 87)
3.5 1.847 1.961 -0.5 1.8 2.01
ML : 1.8 4.83 3.32
input odds before game started (1, 87)
3.5 1.869 1.943 -0.5 1.826 1.98
ML : 1.826 4.75 3.27
input odds before game started (1, 105)
1.5 1.51 2.55 -0.5 1.854 1.99
ML : 1.854 3.24 4.89
input odds before game started (1, 87)
3.5 1.862 1.952 -0.5 1.684 2.16
ML : 1.684 5.04 3.63
input odds before game started (1, 111)
2.5 2.09 1.719 -0.5 2.02 1.813
ML : 2.02 3.42 3.79
input odds before game started (1, 87)
3.5 1.862 1.952 -0.5 1.8 2.0
ML : 1.714 4.95 3.57
input odds before game started (1, 81)
2.5 1.699 2.14 0.5 2.21 1.653
ML : 4.76 3.95 1.653
input odds before game started (1, 99)
2.5 2.03 1.769 -0.5 1.781 2.05
ML : 1.787 3.7 4.47
input odds before game started (1, 87)
3.5 1.862 1.952 -0.5 1.751 2.07
ML : 1.751 4.86 3.46
input odds before game started (1, 111)
2.5 2.01 1.787 -0.5 1.724 2.12
M

ML : 2.95 3.19 2.58
input odds before game started (1, 111)
2.5 2.42 1.588 0.5 1.537 2.58
ML : 2.95 3.19 2.58
input odds before game started (1, 87)
2.5 2.25 1.645 -0.5 2.11 1.751
ML : 2.11 3.25 3.72
input odds before game started (1, 111)
2.5 1.826 2.0 -0.5 2.07 1.781
ML : 2.07 3.5 3.53
input odds before game started (1, 111)
2.5 2.42 1.588 0.5 1.526 2.61
ML : 2.9 3.19 2.61
input odds before game started (1, 87)
2.5 1.552 2.41 -0.5 2.56 1.495
ML : 2.56 3.48 2.62
input odds before game started (1, 111)
2.5 2.42 1.588 0.5 1.502 2.68
ML : 2.83 3.19 2.68
input odds before game started (1, 81)
3.5 1.757 2.08 2.5 1.735 2.13
ML : 17.55 8.9 1.12
input odds before game started (1, 111)
2.5 2.42 1.588 -0.5 2.78 1.471
ML : 2.78 3.19 2.72
input odds before game started (1, 87)
2.5 1.769 2.07 -1.5 2.47 1.552
ML : 1.438 4.29 7.53
input odds before game started (1, 87)
2.5 1.628 2.26 -0.5 1.709 2.13
ML : 1.709 4.1 4.21
input odds before game started (1, 87)
2.5 1.833 2.0 -1.5 2.5 1.543
ML : 1.446 4.

...

3.5 1.781 2.02 -0.5 1.632 2.25
ML : 1.632 4.41 4.37
input odds before game started (1, 81)
3.5 1.781 2.05 -1.5 1.609 2.35
ML : 1.237 6.99 8.76
input odds before game started (1, 111)
2.5 1.877 1.99 -0.5 2.16 1.74
ML : 2.16 3.63 3.3
input odds before game started (1, 81)
3.5 1.793 2.0 -0.5 1.671 2.18
ML : 1.671 4.3 4.22
input odds before game started (1, 87)
3.5 1.775 2.02 -0.5 1.709 2.13
ML : 1.709 4.24 4.08
input odds before game started (1, 87)
3.5 1.781 2.01 -0.5 1.74 2.08
ML : 1.74 4.14 4.0
input odds before game started (1, 87)
3.5 1.769 2.03 -0.5 1.709 2.12
ML : 1.709 4.21 4.09
input odds before game started (1, 81)
3.5 1.826 2.0 -1.5 1.625 2.32
ML : 1.24 6.94 8.68
input odds before game started (1, 87)
3.5 1.787 2.01 -0.5 1.819 1.99
ML : 1.819 3.95 3.84
input odds before game started (1, 87)
3.5 1.934 1.892 0.5 1.806 2.04
ML : 3.1 4.19 2.04
input odds before game started (1, 87)
3.5 1.787 2.01 -0.5 1.787 2.03
ML : 1.787 4.02 3.91
input odds before game started (1, 87)
3.5 1.934 

ML : 2.13 3.99 3.01
input odds before game started (1, 87)
3.5 1.934 1.9 -0.5 2.06 1.787
ML : 2.06 4.05 3.13
input odds before game started (1, 87)
3.5 1.934 1.9 -0.5 2.08 1.775
ML : 2.08 4.08 3.08
input odds before game started (1, 81)
3.5 1.854 1.97 -1.5 1.645 2.28
ML : 1.248 6.83 8.48
input odds before game started (1, 87)
2.5 1.518 2.53 -0.5 1.709 2.17
ML : 1.709 4.68 3.89
input odds before game started (1, 87)
3.5 2.08 1.751 -0.5 1.581 2.41
ML : 1.581 5.43 4.14
input odds before game started (1, 87)
3.5 1.943 1.884 -0.5 2.08 1.775
ML : 2.08 4.08 3.08
input odds before game started (1, 87)
3.5 1.934 1.892 0.5 1.787 2.06
ML : 3.14 4.04 2.06
input odds before game started (1, 87)
3.5 2.19 1.68 0.5 1.584 2.4
ML : 2.58 4.05 2.4
input odds before game started (1, 87)
3.5 1.961 1.869 -0.5 2.08 1.775
ML : 2.08 4.08 3.08
input odds before game started (1, 123)
2.5 1.757 2.17 0.5 1.704 2.27
ML : 3.26 3.53 2.27
input odds before game started (1, 87)
2.5 1.523 2.52 -0.5 1.746 2.11
ML : 1.746 

2.5 2.16 1.704 -0.5 1.529 2.53
ML : 1.529 4.11 6.16
input odds before game started (1, 87)
2.5 2.16 1.699 -0.5 1.534 2.52
ML : 1.534 4.08 6.14
input odds before game started (1, 42)
4.5 1.826 1.934 3.5 2.04 1.74
next "['HOME' 'DRAW' 'AWAY'] not in index"
input odds before game started (1, 123)
2.5 1.724 2.21 0.5 1.724 2.24
ML : 3.28 3.59 2.24
input odds before game started (1, 123)
2.5 1.724 2.21 0.5 1.729 2.23
ML : 3.3 3.59 2.23
input odds before game started (1, 123)
2.5 1.724 2.22 0.5 1.746 2.21
ML : 3.34 3.6 2.21
input odds before game started (1, 123)
2.5 1.729 2.21 0.5 1.719 2.25
ML : 3.31 3.54 2.25
input odds before game started (1, 36)
4.5 1.793 1.97 3.5 2.12 1.689
next "['HOME' 'DRAW' 'AWAY'] not in index"
input odds before game started (1, 42)
4.5 1.763 2.0 3.5 2.18 1.649
next "['HOME' 'DRAW' 'AWAY'] not in index"
input odds before game started (1, 42)
4.5 1.763 2.0 3.5 2.24 1.613
next "['HOME' 'DRAW' 'AWAY'] not in index"
input odds before game started (1, 123)
2.5 1.729 2.2

ML : 2.53 3.5 2.71
input odds before game started (1, 87)
2.5 1.97 1.854 -0.5 2.43 1.568
ML : 2.43 3.5 2.82
input odds before game started (1, 87)
2.5 1.97 1.854 -0.5 2.5 1.543
ML : 2.5 3.5 2.75
input odds before game started (1, 87)
2.5 1.97 1.862 -0.5 2.42 1.574
ML : 2.42 3.54 2.81
input odds before game started (1, 87)
2.5 1.97 1.854 -0.5 2.47 1.555
ML : 2.47 3.49 2.79
input odds before game started (1, 87)
2.5 1.98 1.847 -0.5 2.45 1.561
ML : 2.45 3.44 2.84
input odds before game started (1, 87)
2.5 1.98 1.847 -0.5 2.44 1.564
ML : 2.44 3.43 2.86
input odds before game started (1, 87)
2.5 1.98 1.847 -0.5 2.41 1.578
ML : 2.41 3.43 2.9
input odds before game started (1, 87)
2.5 1.98 1.847 -0.5 2.35 1.606
ML : 2.35 3.46 2.98
input odds before game started (1, 87)
2.5 1.97 1.854 -0.5 2.28 1.641
ML : 2.28 3.5 3.06
input odds before game started (1, 87)
2.5 1.98 1.854 -0.5 2.31 1.628
ML : 2.31 3.48 3.03
input odds before game started (1, 87)
2.5 1.97 1.854 -0.5 2.27 1.653
ML : 2.27 3.51 3.

ML : 2.01 3.7 3.76
input odds before game started (1, 87)
2.5 2.15 1.709 -0.5 2.22 1.675
ML : 2.22 3.14 3.54
input odds before game started (1, 87)
2.5 1.98 1.781 -0.5 1.952 1.826
ML : 1.952 3.6 3.56
input odds before game started (1, 87)
2.5 1.806 2.03 -1.5 2.65 1.487
ML : 1.49 4.36 6.28
input odds before game started (1, 87)
2.5 1.8 2.03 -1.5 2.64 1.492
ML : 1.483 4.38 6.33
input odds before game started (1, 87)
2.5 1.8 2.03 -1.5 2.62 1.497
ML : 1.478 4.4 6.37
input odds before game started (1, 117)
2.5 1.884 2.0 -0.5 1.97 1.925
ML : 1.97 3.65 3.97
input odds before game started (1, 87)
2.5 2.16 1.704 -0.5 1.54 2.51
ML : 1.54 3.97 6.35
input odds before game started (1, 87)
2.5 2.16 1.699 -0.5 1.552 2.47
ML : 1.552 3.93 6.26
input odds before game started (1, 87)
2.5 2.16 1.699 -0.5 1.564 2.38
ML : 1.552 3.93 6.26
input odds before game started (1, 87)
2.5 2.17 1.694 -0.5 1.564 2.44
ML : 1.564 3.88 6.2
input odds before game started (1, 81)
2.5 1.854 1.98 1.5 1.52 2.55
ML : 6.28 4.09

In [85]:
# total errors
book = 'pinnacle'

print(book)
print('all total errors')
print("min error:",  np.array(total_errors[book]).min())
print("max error:",  np.array(total_errors[book]).max())
print("mean error:",  np.array(total_errors[book]).mean())
print("std error:",  np.array(total_errors[book]).std())
print("25% error:",  np.percentile(total_errors[book], q=25))
print("50% error:",  np.percentile(total_errors[book], q=50))
print("75% error:",  np.percentile(total_errors[book], q=75))
print("95% error:",  np.percentile(total_errors[book], q=95))
print("99% error:",  np.percentile(total_errors[book], q=99))
print()

print('int total errors')
print("min error:",  np.array(int_total_errors[book]).min())
print("max error:",  np.array(int_total_errors[book]).max())
print("mean error:",  np.array(int_total_errors[book]).mean())
print("std error:",  np.array(int_total_errors[book]).std())
print("25% error:",  np.percentile(int_total_errors[book], q=25))
print("50% error:",  np.percentile(int_total_errors[book], q=50))
print("75% error:",  np.percentile(int_total_errors[book], q=75))
print("95% error:",  np.percentile(int_total_errors[book], q=95))
print("99% error:",  np.percentile(int_total_errors[book], q=99))
print()

print('half total errors')
print("min error:",  np.array(half_total_errors[book]).min())
print("max error:",  np.array(half_total_errors[book]).max())
print("mean error:",  np.array(half_total_errors[book]).mean())
print("std error:",  np.array(half_total_errors[book]).std())
print("25% error:",  np.percentile(half_total_errors[book], q=25))
print("50% error:",  np.percentile(half_total_errors[book], q=50))
print("75% error:",  np.percentile(half_total_errors[book], q=75))
print("95% error:",  np.percentile(half_total_errors[book], q=95))
print("99% error:",  np.percentile(half_total_errors[book], q=99))
print()

print('quarter total errors')
print("min error:",  np.array(quarter_total_errors[book]).min())
print("max error:",  np.array(quarter_total_errors[book]).max())
print("mean error:",  np.array(quarter_total_errors[book]).mean())
print("std error:",  np.array(quarter_total_errors[book]).std())
print("25% error:",  np.percentile(quarter_total_errors[book], q=25))
print("50% error:",  np.percentile(quarter_total_errors[book], q=50))
print("75% error:",  np.percentile(quarter_total_errors[book], q=75))
print("95% error:",  np.percentile(quarter_total_errors[book], q=95))
print("99% error:",  np.percentile(quarter_total_errors[book], q=99))
print()

pinnacle
all total errors
min error: 0.0
max error: 0.052053973957065125
mean error: 0.0038074844211809645
std error: 0.004802866487847228
25% error: 0.0004940409596493334
50% error: 0.0021344341732928984
75% error: 0.005354624701582877
95% error: 0.013234211256790496
99% error: 0.021504430137153183

int total errors
min error: 8.538199607333041e-07
max error: 0.039353313611464213
mean error: 0.005121779299694548
std error: 0.0048226232528030604
25% error: 0.0016114579656148348
50% error: 0.0038285031743847397
75% error: 0.0071066368538935665
95% error: 0.014490619002506016
99% error: 0.022710410899996057

half total errors
min error: 0.0
max error: 0.052053973957065125
mean error: 0.003659173154773242
std error: 0.005423844549656444
25% error: 6.445925450848655e-06
50% error: 0.0011661546723840122
75% error: 0.0056270971263505665
95% error: 0.014781688377044543
99% error: 0.02293058659752789

quarter total errors
min error: 8.360972747478712e-09
max error: 0.04642368000740804
mean err

In [86]:
# total home errors
book = 'pinnacle'

print(book)
print('all total home errors')
print("min error:",  np.array(total_1_errors[book]).min())
print("max error:",  np.array(total_1_errors[book]).max())
print("mean error:",  np.array(total_1_errors[book]).mean())
print("std error:",  np.array(total_1_errors[book]).std())
print("25% error:",  np.percentile(total_1_errors[book], q=25))
print("50% error:",  np.percentile(total_1_errors[book], q=50))
print("75% error:",  np.percentile(total_1_errors[book], q=75))
print("95% error:",  np.percentile(total_1_errors[book], q=95))
print("99% error:",  np.percentile(total_1_errors[book], q=99))
print()

try:
    print('int total home errors')
    print("min error:",  np.array(int_total_1_errors[book]).min())
    print("max error:",  np.array(int_total_1_errors[book]).max())
    print("mean error:",  np.array(int_total_1_errors[book]).mean())
    print("std error:",  np.array(int_total_1_errors[book]).std())
    print("25% error:",  np.percentile(int_total_1_errors[book], q=25))
    print("50% error:",  np.percentile(int_total_1_errors[book], q=50))
    print("75% error:",  np.percentile(int_total_1_errors[book], q=75))
    print("95% error:",  np.percentile(int_total_1_errors[book], q=95))
    print("99% error:",  np.percentile(int_total_1_errors[book], q=99))
    print()
except:
    pass

print('half total home errors')
print("min error:",  np.array(half_total_1_errors[book]).min())
print("max error:",  np.array(half_total_1_errors[book]).max())
print("mean error:",  np.array(half_total_1_errors[book]).mean())
print("std error:",  np.array(half_total_1_errors[book]).std())
print("25% error:",  np.percentile(half_total_1_errors[book], q=25))
print("50% error:",  np.percentile(half_total_1_errors[book], q=50))
print("75% error:",  np.percentile(half_total_1_errors[book], q=75))
print("95% error:",  np.percentile(half_total_1_errors[book], q=95))
print("99% error:",  np.percentile(half_total_1_errors[book], q=99))
print()

try:
    print('quarter total home errors')
    print("min error:",  np.array(quarter_total_1_errors[book]).min())
    print("max error:",  np.array(quarter_total_1_errors[book]).max())
    print("mean error:",  np.array(quarter_total_1_errors[book]).mean())
    print("std error:",  np.array(quarter_total_1_errors[book]).std())
    print("25% error:",  np.percentile(quarter_total_1_errors[book], q=25))
    print("50% error:",  np.percentile(quarter_total_1_errors[book], q=50))
    print("75% error:",  np.percentile(quarter_total_1_errors[book], q=75))
    print("95% error:",  np.percentile(quarter_total_1_errors[book], q=95))
    print("99% error:",  np.percentile(quarter_total_1_errors[book], q=99))
    print()
except:
    pass

pinnacle
all total home errors
min error: 2.6851914259884424e-06
max error: 0.14943313582430467
mean error: 0.010818610324273701
std error: 0.01297026513707801
25% error: 0.0032445219855432583
50% error: 0.007289873026607607
75% error: 0.013797417921040611
95% error: 0.03385616971596242
99% error: 0.06735923799166543

int total home errors
min error: 1.0824711311530066e-05
max error: 0.14943313582430467
mean error: 0.011774524084342477
std error: 0.013619138033616088
25% error: 0.004124236275908688
50% error: 0.008591431364682656
75% error: 0.015569108700044354
95% error: 0.02807975620119612
99% error: 0.07949049451285736

half total home errors
min error: 2.6851914259884424e-06
max error: 0.13068351260511957
mean error: 0.010097478894489373
std error: 0.012409787084813285
25% error: 0.0027895453718617957
50% error: 0.006329581235199111
75% error: 0.012169456676600313
95% error: 0.03616023787099167
99% error: 0.058697494306976766

quarter total home errors


In [57]:
# # total away errors
# book = 'pinnacle'

# print(book)
# print('all total away errors')
# print("min error:",  np.array(total_2_errors[book]).min())
# print("max error:",  np.array(total_2_errors[book]).max())
# print("mean error:",  np.array(total_2_errors[book]).mean())
# print("std error:",  np.array(total_2_errors[book]).std())
# print("25% error:",  np.percentile(total_2_errors[book], q=25))
# print("50% error:",  np.percentile(total_2_errors[book], q=50))
# print("75% error:",  np.percentile(total_2_errors[book], q=75))
# print("95% error:",  np.percentile(total_2_errors[book], q=95))
# print("99% error:",  np.percentile(total_2_errors[book], q=99))
# print()

# try:
#     print('int total away errors')
#     print("min error:",  np.array(int_total_2_errors[book]).min())
#     print("max error:",  np.array(int_total_2_errors[book]).max())
#     print("mean error:",  np.array(int_total_2_errors[book]).mean())
#     print("std error:",  np.array(int_total_2_errors[book]).std())
#     print("25% error:",  np.percentile(int_total_2_errors[book], q=25))
#     print("50% error:",  np.percentile(int_total_2_errors[book], q=50))
#     print("75% error:",  np.percentile(int_total_2_errors[book], q=75))
#     print("95% error:",  np.percentile(int_total_2_errors[book], q=95))
#     print("99% error:",  np.percentile(int_total_2_errors[book], q=99))
#     print()
# except:
#     pass

# print('half total away errors')
# print("min error:",  np.array(half_total_2_errors[book]).min())
# print("max error:",  np.array(half_total_2_errors[book]).max())
# print("mean error:",  np.array(half_total_2_errors[book]).mean())
# print("std error:",  np.array(half_total_2_errors[book]).std())
# print("25% error:",  np.percentile(half_total_2_errors[book], q=25))
# print("50% error:",  np.percentile(half_total_2_errors[book], q=50))
# print("75% error:",  np.percentile(half_total_2_errors[book], q=75))
# print("95% error:",  np.percentile(half_total_2_errors[book], q=95))
# print("99% error:",  np.percentile(half_total_2_errors[book], q=99))
# print()

# try:
#     print('quarter total away errors')
#     print("min error:",  np.array(quarter_total_2_errors[book]).min())
#     print("max error:",  np.array(quarter_total_2_errors[book]).max())
#     print("mean error:",  np.array(quarter_total_2_errors[book]).mean())
#     print("std error:",  np.array(quarter_total_2_errors[book]).std())
#     print("25% error:",  np.percentile(quarter_total_2_errors[book], q=25))
#     print("50% error:",  np.percentile(quarter_total_2_errors[book], q=50))
#     print("75% error:",  np.percentile(quarter_total_2_errors[book], q=75))
#     print("95% error:",  np.percentile(quarter_total_2_errors[book], q=95))
#     print("99% error:",  np.percentile(quarter_total_2_errors[book], q=99))
#     print()
# except:
#     pass

pinnacle
all total away errors
min error: 1.2112573287148543e-05
max error: 0.45209701787188505
mean error: 0.11547221799305855
std error: 0.08095854053780545
25% error: 0.05142736001029491
50% error: 0.10043020909194367
75% error: 0.16305353707834297
95% error: 0.2782871133782894
99% error: 0.32297488920773665

int total away errors
min error: 0.00014271407771160316
max error: 0.41235994862937864
mean error: 0.12993385424067547
std error: 0.0840205141350659
25% error: 0.06078078970115702
50% error: 0.12104111270206602
75% error: 0.1871451428861759
95% error: 0.2849661218871924
99% error: 0.32247182847257383

half total away errors
min error: 1.2112573287148543e-05
max error: 0.45209701787188505
mean error: 0.10453507523998867
std error: 0.07677531046729334
25% error: 0.04520919631571303
50% error: 0.08887826493064824
75% error: 0.14731319758321157
95% error: 0.2604326305773094
99% error: 0.32463774618794483

quarter total away errors


In [87]:
# handicap errors
book = 'pinnacle'

print(book)
print('all hdp errors')
print("min error:",  np.array(hdp_1_errors[book]).min())
print("max error:",  np.array(hdp_1_errors[book]).max())
print("mean error:",  np.array(hdp_1_errors[book]).mean())
print("std error:",  np.array(hdp_1_errors[book]).std())
print("25% error:",  np.percentile(hdp_1_errors[book], q=25))
print("50% error:",  np.percentile(hdp_1_errors[book], q=50))
print("75% error:",  np.percentile(hdp_1_errors[book], q=75))
print("95% error:",  np.percentile(hdp_1_errors[book], q=95))
print("99% error:",  np.percentile(hdp_1_errors[book], q=99))
print()

try:
    print('int hdp errors')
    print("min error:",  np.array(int_hdp_1_errors[book]).min())
    print("max error:",  np.array(int_hdp_1_errors[book]).max())
    print("mean error:",  np.array(int_hdp_1_errors[book]).mean())
    print("std error:",  np.array(int_hdp_1_errors[book]).std())
    print("25% error:",  np.percentile(int_hdp_1_errors[book], q=25))
    print("50% error:",  np.percentile(int_hdp_1_errors[book], q=50))
    print("75% error:",  np.percentile(int_hdp_1_errors[book], q=75))
    print("95% error:",  np.percentile(int_hdp_1_errors[book], q=95))
    print("99% error:",  np.percentile(int_hdp_1_errors[book], q=99))
    print()
except:
    pass

print('half hdp errors')
print("min error:",  np.array(half_hdp_1_errors[book]).min())
print("max error:",  np.array(half_hdp_1_errors[book]).max())
print("mean error:",  np.array(half_hdp_1_errors[book]).mean())
print("std error:",  np.array(half_hdp_1_errors[book]).std())
print("25% error:",  np.percentile(half_hdp_1_errors[book], q=25))
print("50% error:",  np.percentile(half_hdp_1_errors[book], q=50))
print("75% error:",  np.percentile(half_hdp_1_errors[book], q=75))
print("95% error:",  np.percentile(half_hdp_1_errors[book], q=95))
print("99% error:",  np.percentile(half_hdp_1_errors[book], q=99))
print()

try:
    print('quarter hdp errors')
    print("min error:",  np.array(quarter_hdp_1_errors[book]).min())
    print("max error:",  np.array(quarter_hdp_1_errors[book]).max())
    print("mean error:",  np.array(quarter_hdp_1_errors[book]).mean())
    print("std error:",  np.array(quarter_hdp_1_errors[book]).std())
    print("25% error:",  np.percentile(quarter_hdp_1_errors[book], q=25))
    print("50% error:",  np.percentile(quarter_hdp_1_errors[book], q=50))
    print("75% error:",  np.percentile(quarter_hdp_1_errors[book], q=75))
    print("95% error:",  np.percentile(quarter_hdp_1_errors[book], q=95))
    print("99% error:",  np.percentile(quarter_hdp_1_errors[book], q=99))
    print()
except:
    pass

pinnacle
all hdp errors
min error: 0.0
max error: 0.15978291079341245
mean error: 0.0086645535504883
std error: 0.010417340898946992
25% error: 0.0011321834329837133
50% error: 0.00551265191417083
75% error: 0.013343892521578138
95% error: 0.026465554275509874
99% error: 0.03973496982433567

int hdp errors
min error: 0.0
max error: 0.06480547446454221
mean error: 0.008787902850290066
std error: 0.009816685665847
25% error: 0.0010528698475691378
50% error: 0.0053389423291489435
75% error: 0.013863265797513263
95% error: 0.02749897604231189
99% error: 0.041832165532060955

half hdp errors
min error: 0.0
max error: 0.15978291079341245
mean error: 0.008571193582817332
std error: 0.010872403457103571
25% error: 0.001231516090930529
50% error: 0.005646526117489087
75% error: 0.013029676129119455
95% error: 0.025312818527267175
99% error: 0.03838434799479713

quarter hdp errors
min error: 2.1315699549884215e-07
max error: 0.027735290763189757
mean error: 0.007416811336190001
std error: 0.0081

In [88]:
# ml errors
book = 'pinnacle'


print(book)
print('ml errors')
print("min error:",  np.array(ml_errors[book]).min())
print("max error:",  np.array(ml_errors[book]).max())
print("mean error:",  np.array(ml_errors[book]).mean())
print("std error:",  np.array(ml_errors[book]).std())
print("25% error:",  np.percentile(ml_errors[book], q=25))
print("50% error:",  np.percentile(ml_errors[book], q=50))
print("75% error:",  np.percentile(ml_errors[book], q=75))
print("95% error:",  np.percentile(ml_errors[book], q=95))
print("99% error:",  np.percentile(ml_errors[book], q=99))
print()

pinnacle
ml errors
min error: 1.4189422270494134e-07
max error: 0.20610990553722114
mean error: 0.011991827136291825
std error: 0.011602663639408033
25% error: 0.003605167187298343
50% error: 0.008966296806272367
75% error: 0.018263214546844764
95% error: 0.03032567894354172
99% error: 0.04286090849030577

